In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from sklearn.base import BaseEstimator
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


from xgboost import XGBRegressor, XGBClassifier

import optuna

/home/niklas/Documents/projects/kicktipp-betbot/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TRIALS = 1

In [3]:
df_matches = pd.read_csv("matches.csv")

## Baseline Models

In [ ]:
class StaticEstimator(BaseEstimator):
    def __init__(self, result) -> None:
        super().__init__()
        self.result = result

    def fit(self, X, y):
        pass

    def predict(self, X):
        results = np.zeros((len(X), 2))
        results[:] = self.result
        return results

class RandomEstimator(BaseEstimator):
    
    def fit(self, X, y):
        df_result_counts = df_matches[["team1Goals", "team2Goals"]].value_counts().reset_index()
        df_result_counts["p"] = df_result_counts["count"] / len(df_matches)
        self.df_result_counts = df_result_counts

    def predict(self, X):
        df_result_counts = self.df_result_counts
        result_indices = np.random.choice(df_result_counts.index, len(X), p=df_result_counts["p"].values)
        return df_result_counts.loc[result_indices, ["team1Goals", "team2Goals"]].values

In [ ]:
from utils import cross_val_splits, kicktipp_scoring


def evaluate_simple_models(scoring):
    splits = cross_val_splits(df_matches)
    labels = df_matches[["team1Goals", "team2Goals"]].values
    static_results = [[0, 0], [1, 1], [1, 0], [0, 1]]
    static_result_scores = []
    for result in static_results:
        scores = cross_val_score(StaticEstimator(result), df_matches, labels, cv=splits, scoring=scoring)
        static_result_scores.append(scores)
    static_result_scores = np.array(static_result_scores)

    random_scores = cross_val_score(RandomEstimator(), df_matches, labels, cv=splits, scoring=scoring)

    all_scores = np.vstack((static_result_scores, random_scores))
    labels = [f"{res[0]}:{res[1]}" for res in static_results] + ["random"]
    for score, label in zip(all_scores, labels):
        print(f"{label}: {score.mean():.2f} +- {score.std():.2f}")

    fig, axs = plt.subplots(ncols=2, figsize=(12, 4))
    split_test_seasons = [df_matches.iloc[split[1][0]]["leagueSeason"] for  split in splits]
    axs[0].plot(all_scores.T, label=labels)
    axs[0].set_xticks(range(len(split_test_seasons)), split_test_seasons, rotation=90, ha='center')
    axs[0].set_xlabel("Test season")
    axs[0].set_ylabel("Score")
    axs[0].legend()
    axs[0].set_title("Cross-validation scores")

    axs[1].boxplot(all_scores.T)
    axs[1].set_xlabel("Strategy")
    axs[1].set_xticks(range(1, len(labels) +1), labels, rotation=45, ha="right")
    fig.tight_layout()

evaluate_simple_models(scoring=kicktipp_scoring)

# Matches with team context

In [ ]:
df_full = pd.read_csv("matches_with_context.csv", index_col="id")
df_full.tail()

In [ ]:
X_all = df_full.drop(columns=["team1Goals", "team2Goals", "result", "matchDay", "result", "leagueName", "leagueID", "date", "team1Short", "team1Name", "team2Short", "team2Name", "host_team", "guest_team"])
y_all = df_full[["team1Goals", "team2Goals"]].values
print(X_all.columns)

In [ ]:
class RoundingEstimator(BaseEstimator):
    def __init__(self, regressor, **kwargs) -> None:
        super().__init__()
        self.regressor = regressor
        self.regressor.set_params(**kwargs)
    
    def fit(self, X, y):
        self.regressor.fit(X, y)
    
    def predict(self, X):
        raw = self.regressor.predict(X)
        return np.round(raw)
    
    def get_params(self, deep = False):
        params = self.regressor.get_params(deep)
        params["regressor"] = self.regressor
        return params

    def set_params(self, **params):
        self.regressor.set_params(**params)
        return self

In [ ]:
splits = cross_val_splits(X_all)
def objective(trial):
    rf_criterion = trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse", "poisson"])
    rf_max_depth = trial.suggest_int('max_depth', 2, 15)
    rf_n_estimators = trial.suggest_int('n_estimators', 2, 200)
    rf_min_samples_split = trial.suggest_int('min_samples_split', 2, 100)
    estimator = RoundingEstimator(RandomForestRegressor(
        n_estimators=rf_n_estimators, 
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
        n_jobs=-1
    ))
    scores = cross_val_score(estimator, X_all, y_all, cv=splits, verbose=1, scoring=kicktipp_scoring)
    return np.mean(scores)

rf_study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name="rf", load_if_exists=True)
rf_study.optimize(objective, n_trials=TRIALS)
print(rf_study.best_value, rf_study.best_params)

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor

def objective(trial):
    algo = trial.suggest_categorical("algo", ["xgb", "lgbm", "gbr"])
    n_estimators = trial.suggest_int("n_estimators", 20, 600)
    max_depth = trial.suggest_int("max_depth", 2, 15)
    reg_lambda=trial.suggest_float("lambda", 0.0, 2.0)
    reg_alpha=trial.suggest_float("alpha", 0.0, 2.0)
    learning_rate = trial.suggest_float("learning_rate", 0.0, 0.5)
    num_leaves=trial.suggest_int("num_leaves", 2, 31)
    min_child_weight = trial.suggest_float("min_child_weight", 0, 15)
    if algo == "xgb":
        estimator = RoundingEstimator(XGBRegressor(
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            max_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=trial.suggest_categorical("objective-xgb", ["reg:squarederror", "reg:squaredlogerror", "reg:absoluteerror"]),
        ))
    elif algo == "lgbm":
        estimator = RoundingEstimator(MultiOutputRegressor(LGBMRegressor(
            verbose=-1,
            n_jobs=-1,
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=trial.suggest_categorical("objective-lgbm", ["rmse", "mae", "poisson", "mape"]),
        )))
    elif algo == "gbr":
        estimator = RoundingEstimator(MultiOutputRegressor(GradientBoostingRegressor(
            loss=trial.suggest_categorical("objective-gbr", ["squared_error", "absolute_error", "huber"]),
            alpha=trial.suggest_float("huber-alpha", 0.0, 0.99),
            learning_rate=learning_rate,
            n_estimators = n_estimators,
            subsample=trial.suggest_float("subsample", 0.0, 1.0),
            max_depth = max_depth,
        )))

    scores = cross_val_score(estimator, X_all, y_all, cv=splits, verbose=1, scoring=kicktipp_scoring)
    return np.mean(scores)

gb_study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name="gb", load_if_exists=True)
gb_study.optimize(objective, n_trials=TRIALS)
print(gb_study.best_value, gb_study.best_params)

In [ ]:
def build_gb_model(study, rounder=RoundingEstimator):
    params = study.best_params
    algo = params["algo"]
    n_estimators = params["n_estimators"]
    max_depth = params["max_depth"]
    reg_lambda=params["lambda"]
    reg_alpha=params["alpha"]
    learning_rate = params["learning_rate"]
    num_leaves=params["num_leaves"]
    min_child_weight = params["min_child_weight"]
    if algo == "xgb":
        estimator = rounder(XGBRegressor(
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            max_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=params["objective-xgb"],
        ))
    elif algo == "lgbm":
        estimator = rounder(MultiOutputRegressor(LGBMRegressor(
            verbose=-1,
            n_jobs=-1,
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=params["objective-lgbm"],
        )))
    return estimator
    

In [ ]:

models = {
    "baseline": StaticEstimator([1, 0]),
    "rf": RoundingEstimator(RandomForestRegressor(
        criterion=rf_study.best_params['criterion'], 
        max_depth=rf_study.best_params['max_depth'], 
        n_estimators=rf_study.best_params['n_estimators'], 
        min_samples_split=rf_study.best_params['min_samples_split']
    )),
    "gb": build_gb_model(gb_study)
}

all_scores = []

for model in models.values():
    print(f"Scoring {model}")
    scores = cross_val_score(model, X_all, y_all, cv=splits, verbose=1, scoring=kicktipp_scoring)
    all_scores.append(scores)

all_scores = np.array(all_scores)
labels = models.keys()

fig, axs = plt.subplots(ncols=2, figsize=(12, 4))
split_test_seasons = [df_matches.iloc[split[1][0]]["leagueSeason"] for  split in splits]
axs[0].plot(all_scores.T, label=labels)
axs[0].set_xticks(range(len(split_test_seasons)), split_test_seasons, rotation=90, ha='center')
axs[0].set_xlabel("Test season")
axs[0].set_ylabel("Score")
axs[0].legend()
axs[0].set_title("Cross-validation scores")

axs[1].boxplot(all_scores.T)
axs[1].set_xlabel("Strategy")
axs[1].set_xticks(range(1, len(labels) +1), labels, rotation=90, ha="center")
fig.tight_layout()
!


In [ ]:
model = build_gb_model(gb_study)
train_idx, test_idx = splits[-2]
X_train, X_test = X_all.iloc[train_idx], X_all.iloc[test_idx]
y_train, y_test = y_all[train_idx], y_all[test_idx]
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_train_pred_raw = model.regressor.predict(X_train)

df_full_train = df_full.iloc[train_idx].copy()
df_full_train["team1Goals_pred"] = y_train_pred[:, 0]
df_full_train["team2Goals_pred"] = y_train_pred[:, 1]
df_full_train["team1Goals_pred_raw"] = y_train_pred_raw[:, 0]
df_full_train["team2Goals_pred_raw"] = y_train_pred_raw[:, 1]
df_full_train[["leagueSeason", "matchDayID", "team1Short", "team2Short", "team1Goals", "team2Goals", "team1Goals_pred", "team2Goals_pred"]]

In [ ]:
df_full_train[["team1Goals_pred", "team2Goals_pred"]].value_counts()

# Classification

In [ ]:
from sklearn.preprocessing import LabelEncoder


class ClassifierEstimator(BaseEstimator):
    def __init__(self, classifier, max_goals=3, **kwargs) -> None:
        super().__init__()
        self.classifier = classifier
        self.classifier.set_params(**kwargs)
        self.target_encoder = LabelEncoder()
        self.max_goals = max_goals

    @staticmethod
    def clip_results(results: np.ndarray, max_goals=3):
        results = results.copy()
        match_goals = results.sum(axis=1)
        for i, res in enumerate(results):
            while match_goals[i] > max_goals:
                results[i, 0] = np.max((0, results[i, 0] - 1))
                results[i, 1] = np.max((0, results[i, 1] - 1))
                match_goals[i] = results[i, 0] + results[i, 1]
        return results.astype(results.dtype)
    
    def fit(self, X, y):
        y = ClassifierEstimator.clip_results(y, self.max_goals)
        results = [f"{a[0]}:{a[1]}" for a in y]
        self.target_encoder.fit(results)
        results_encoded = self.target_encoder.transform(results)
        self.classifier.fit(X, results_encoded)
    
    def predict(self, X):
        raw = self.classifier.predict(X)
        results = self.target_encoder.inverse_transform(raw)
        y = [[int(a.split(":")[0]), int(a.split(":")[1])] for a in results]
        return np.array(y)
    
    def get_params(self, deep = False):
        params = self.classifier.get_params(deep)
        params["classifier"] = self.classifier
        return params

    def set_params(self, **params):
        self.classifier.set_params(**params)
        return self

In [ ]:
def objective(trial):
    rf_criterion = trial.suggest_categorical("criterion", ["gini", "log_loss", "entropy"])
    rf_max_depth = trial.suggest_int('max_depth', 2, 15)
    rf_n_estimators = trial.suggest_int('n_estimators', 2, 200)
    rf_min_samples_split = trial.suggest_int('min_samples_split', 2, 100)
    max_goals = trial.suggest_int('max_goals', 2, 5)
    estimator = ClassifierEstimator(RandomForestClassifier(
        n_estimators=rf_n_estimators, 
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
        n_jobs=-1
    ), max_goals=max_goals)
    scores = cross_val_score(estimator, X_all, y_all, cv=splits, verbose=1, scoring=kicktipp_scoring)
    return np.mean(scores)

rf_study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name="rf-classifier", load_if_exists=True)
rf_study.optimize(objective, n_trials=TRIALS)
print(rf_study.best_value, rf_study.best_params)

In [ ]:

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

def objective(trial):
    algo = trial.suggest_categorical("algo", ["xgb", "lgbm"])
    n_estimators = trial.suggest_int("n_estimators", 20, 300)
    max_depth = trial.suggest_int("max_depth", 2, 15)
    reg_lambda=trial.suggest_float("lambda", 0.0, 2.0)
    reg_alpha=trial.suggest_float("alpha", 0.0, 2.0)
    learning_rate = trial.suggest_float("learning_rate", 0.0, 0.5)
    num_leaves=trial.suggest_int("num_leaves", 2, 31)
    min_child_weight = trial.suggest_float("min_child_weight", 0, 15)
    max_goals = trial.suggest_int('max_goals', 2, 5)
    if algo == "xgb":
        estimator = ClassifierEstimator(XGBClassifier(
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            max_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=trial.suggest_categorical("objective-xgb", ["multi:softmax"]),
        ), max_goals=max_goals)
    elif algo == "lgbm":
        estimator = ClassifierEstimator(LGBMClassifier(
            verbose=-1,
            n_jobs=-1,
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=trial.suggest_categorical("objective-lgbm", ["multiclass"]),
        ), max_goals=max_goals)
    elif algo == "gbr":
        estimator = ClassifierEstimator(GradientBoostingClassifier(
            loss=trial.suggest_categorical("objective-gbr", ["log_loss"]),
            alpha=trial.suggest_float("huber-alpha", 0.0, 0.99),
            learning_rate=learning_rate,
            n_estimators = n_estimators,
            subsample=trial.suggest_float("subsample", 0.0, 1.0),
            max_depth = max_depth,
        ), max_goals=max_goals)

    scores = cross_val_score(estimator, X_all, y_all, cv=splits, verbose=1, scoring=kicktipp_scoring)
    return np.mean(scores)

gb_study = optuna.create_study(direction='maximize', storage="sqlite:///db.sqlite3", study_name="gb-classifier", load_if_exists=True)
gb_study.optimize(objective, n_trials=TRIALS)
print(gb_study.best_value, gb_study.best_params)

In [ ]:
def build_gb_model_classifier(study):
    params = study.best_params
    algo = params["algo"]
    n_estimators = params["n_estimators"]
    max_depth = params["max_depth"]
    reg_lambda=params["lambda"]
    reg_alpha=params["alpha"]
    learning_rate = params["learning_rate"]
    num_leaves=params["num_leaves"]
    min_child_weight = params["min_child_weight"]
    max_goals = params["max_goals"]
    if algo == "xgb":
        estimator = ClassifierEstimator(XGBClassifier(
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            max_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=params["objective-xgb"],
        ), max_goals=max_goals)
    elif algo == "lgbm":
        estimator = ClassifierEstimator(LGBMClassifier(
            verbose=-1,
            n_jobs=-1,
            n_estimators = n_estimators,
            max_depth = max_depth,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            min_child_weight=min_child_weight,
            objective=params["objective-lgbm"],
        ), max_goals=max_goals)
    return estimator
    

In [ ]:
models = {
    "baseline": StaticEstimator([1, 0]),
    "rf": ClassifierEstimator(RandomForestClassifier(
        criterion=rf_study.best_params['criterion'], 
        max_depth=rf_study.best_params['max_depth'], 
        n_estimators=rf_study.best_params['n_estimators'], 
        min_samples_split=rf_study.best_params['min_samples_split']
    ), max_goals=rf_study.best_params["max_goals"]),
    "gb": build_gb_model_classifier(gb_study)
}

all_scores = []

for model in models.values():
    print(f"Scoring {model}")
    scores = cross_val_score(model, X_all, y_all, cv=splits, verbose=1, scoring=kicktipp_scoring)
    all_scores.append(scores)

all_scores = np.array(all_scores)
labels = models.keys()

fig, axs = plt.subplots(ncols=2, figsize=(12, 4))
split_test_seasons = [df_matches.iloc[split[1][0]]["leagueSeason"] for  split in splits]
axs[0].plot(all_scores.T, label=labels)
axs[0].set_xticks(range(len(split_test_seasons)), split_test_seasons, rotation=90, ha='center')
axs[0].set_xlabel("Test season")
axs[0].set_ylabel("Score")
axs[0].legend()
axs[0].set_title("Cross-validation scores")

axs[1].boxplot(all_scores.T)
axs[1].set_xlabel("Strategy")
axs[1].set_xticks(range(1, len(labels) +1), labels, rotation=90, ha="center")
fig.tight_layout()
!